In [1]:
import requests
import json
import time

# 官方文档地址
# https://doc2.bitbrowser.cn/jiekou/ben-di-fu-wu-zhi-nan.html

# 此demo仅作为参考使用，以下使用的指纹参数仅是部分参数，完整参数请参考文档

url = "http://127.0.0.1:54345"
headers = {'Content-Type': 'application/json'}


def createBrowser():  # 创建或者更新窗口，指纹参数 browserFingerPrint 如没有特定需求，只需要指定下内核即可，如果需要更详细的参数，请参考文档
    json_data = {
        'name': 'google',  # 窗口名称
        'remark': '',  # 备注
        'proxyMethod': 2,  # 代理方式 2自定义 3 提取IP
        # 代理类型  ['noproxy', 'http', 'https', 'socks5', 'ssh']
        'proxyType': 'noproxy',
        'host': '',  # 代理主机
        'port': '',  # 代理端口
        'proxyUserName': '',  # 代理账号
        "browserFingerPrint": {  # 指纹对象
            'coreVersion': '112'  # 内核版本 112 | 104，建议使用112，注意，win7/win8/winserver 2012 已经不支持112内核了，无法打开
        }
    }

    res = requests.post(f"{url}/browser/update",
                        data=json.dumps(json_data), headers=headers).json()
    browserId = res['data']['id']
    print(browserId)
    return browserId


def updateBrowser():  # 更新窗口，支持批量更新和按需更新，ids 传入数组，单独更新只传一个id即可，只传入需要修改的字段即可，比如修改备注，具体字段请参考文档，browserFingerPrint指纹对象不修改，则无需传入
    json_data = {'ids': ['93672cf112a044f08b653cab691216f0'],
                 'remark': '我是一个备注', 'browserFingerPrint': {}}
    res = requests.post(f"{url}/browser/update/partial",
                        data=json.dumps(json_data), headers=headers).json()
    print(res)


def openBrowser(id):  # 直接指定ID打开窗口，也可以使用 createBrowser 方法返回的ID
    json_data = {"id": f'{id}'}
    res = requests.post(f"{url}/browser/open",
                        data=json.dumps(json_data), headers=headers).json()
    print(res)
    print(res['data']['http'])
    return res


def closeBrowser(id):  # 关闭窗口
    json_data = {'id': f'{id}'}
    requests.post(f"{url}/browser/close",
                  data=json.dumps(json_data), headers=headers).json()


def deleteBrowser(id):  # 删除窗口
    json_data = {'id': f'{id}'}
    print(requests.post(f"{url}/browser/delete",
          data=json.dumps(json_data), headers=headers).json())


In [2]:
browser_id = createBrowser()
openBrowser(browser_id)
time.sleep(1)

6a84de866bea4190bf4c1c7f02a8d584
{'success': True, 'data': {'ws': 'ws://127.0.0.1:9036/devtools/browser/3adb7415-47c9-4b5e-b1d1-6456fa2858c9', 'http': '127.0.0.1:9036', 'coreVersion': '112', 'driver': 'C:\\Users\\yuzhe\\AppData\\Roaming\\BitBrowser\\chromedriver\\112\\chromedriver.exe', 'seq': 13, 'name': 'google', 'remark': '', 'pid': 27168}}
127.0.0.1:9036


In [4]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bit_api import *
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# /browser/open 接口会返回 selenium使用的http地址，以及webdriver的path，直接使用即可
id = '2395deaf0bc246808e73be20c0bb650b'
res = openBrowser(f"{id}")
driverPath = res['data']['driver']
debuggerAddress = res['data']['http']

print(driverPath)
print(debuggerAddress)
# selenium 连接代码
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("debuggerAddress", debuggerAddress)

chrome_service = Service(driverPath)
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)


# 导航到网页
driver.get('https://www.ozon.ru/')
time.sleep(10)


{'success': False, 'msg': '服务调用成功,但没有找到相应数据！'}


KeyError: 'data'

In [15]:
with open ('./ozon.html', 'w') as f:
    f.write(driver.page_source)

In [24]:
frame = driver.find_element(By.XPATH,"/html/body/div[1]/div/div[1]/div/div/iframe")
driver.switch_to.frame(frame)
print("switch to frame success")

switch to frame success


In [25]:
driver.find_element(By.XPATH,"//*[@id='challenge-stage']/div/label/input").click()